## Project Goal
The goal of this project is to implement k-nearest and averaged perceptron from scratch for binary income classification on semi-blind data. The data folder includes training data (5000 examples with labels), dev data (1000 examples with labels) and the semi-blind test data (1000 examples without labels). 

### Importing packages

In [141]:
import numpy as np 
import time

### K-Nearest Algorithm

In [118]:
def data(file):
    data = [s.strip().split(", ") for s in open(file).readlines()]
    numerical_fields = [0,7]
    new_data, perc_targets, knn_targets, num_data = [], [], [], []
    mapping = {} #DO NOT initialize for dev
    for row in data:
        bin_row, numerical_row =[], []
        for j, x in enumerate(row):
            feature = (j,x)
            if j in numerical_fields:
                numerical_row.append(float(x)/50) #normalizes numerical field
            if j == 9:
                knn_targets.append(x)
                if x == ">50K": 
                    perc_targets.append(1)
                else:
                    perc_targets.append(-1)
            else:
                if j not in numerical_fields: # preserves numerical data for knn 
                    if feature not in mapping: 
                        mapping[feature]= len(mapping)
                    if feature in mapping: # INCLUDE for dev instead of if statement above and indent line below
                        bin_row.append(mapping[feature])
        new_data.append(bin_row)
        num_data.append(numerical_row)
    x_dim, y_dim = len(new_data), len(mapping)
    binary_data = np.zeros((x_dim,y_dim))
    for i, row in enumerate(new_data):
        for x in row: 
            binary_data[i][x] =1
    #add bias, see slides 4.13-15
    bin_with_bias = np.concatenate((binary_data, np.ones((binary_data.shape[0],1))), axis=1)
    return np.array(bin_with_bias), np.array(perc_targets), mapping  # remove mapping variable for dev

In [60]:
# Training set, targets, numerical data and mapping
train, train_knn_targets, train_num_data = data('./data/income.train.txt.5k')

In [62]:
train_with_num = np.concatenate((train, train_num_data), axis=1)

In [64]:
# Dev Set 
dev, dev_knn_targets, dev_num_data = data('./data/income.dev.txt')

In [65]:
dev_with_num = np.concatenate((dev, dev_num_data), axis=1)

In [72]:
# blind set, targets, numerical data and mapping
blind, blind_knn_targets, blind_num_data = data('./data/income.test.blind') # blind_knn_target is empty

In [77]:
blind_with_num = np.concatenate((blind, blind_num_data), axis=1)

In [78]:
# Euclidean dev error rate for kNN 
def euc_dev_error(train_set,train_label,dev_set,dev_label,k): 
    predictions=[]
    labels=np.array(train_label)
    for i, test in enumerate(dev_set):
        pos_pred, neg_pred = 0 , 0
        true_label = dev_label[i]
        euc_dist = np.linalg.norm(train_set-test, axis=1)
        nearest = labels[np.argsort(euc_dist)[:k]]
        for neighbor in nearest:
            if neighbor == '>50K': 
                pos_pred += 1
            else: 
                neg_pred += 1
        if pos_pred > neg_pred:
            knn_pred = ">50K"
        else: 
            knn_pred = "<=50K"
        if knn_pred == true_label: 
            pred=1
        else:
            pred=0
        predictions.append(pred)
    dev_error = (len(dev_label)-np.sum(predictions))/len(dev_label)*100
    return f"The euclidean dev error rate is for {k}-NN is {dev_error}%"

In [79]:
# finding optimal k using euclidean distance
for k in range(1,100,10):
    print(euc_dev_error(train_with_num,train_knn_targets,dev_with_num,dev_knn_targets,k))

The euclidean dev error rate is for 1-NN is 22.900000000000002%
The euclidean dev error rate is for 11-NN is 16.3%
The euclidean dev error rate is for 21-NN is 15.9%
The euclidean dev error rate is for 31-NN is 15.4%
The euclidean dev error rate is for 41-NN is 14.6%
The euclidean dev error rate is for 51-NN is 15.299999999999999%
The euclidean dev error rate is for 61-NN is 15.299999999999999%
The euclidean dev error rate is for 71-NN is 15.2%
The euclidean dev error rate is for 81-NN is 15.5%
The euclidean dev error rate is for 91-NN is 15.5%


In [89]:
def blind_predictions(train, labels, blind_rawfile, k):
    for i, new_test in enumerate(blind_with_num):
        pos_pred, neg_pred  = 0 , 0
        euc_dist = np.linalg.norm(train-new_test, axis=1)
        nearest = labels[np.argsort(euc_dist)[:k]]
        for neighbor in nearest:
            if neighbor == '>50K': 
                pos_pred += 1
            else: 
                neg_pred += 1
        if pos_pred > neg_pred:
            knn_pred = ">50K"
        else: 
            knn_pred = "<=50K"
        blind_rawfile[i].append(knn_pred)

In [205]:
raw_blind = [s.strip().split(",") for s in open('./data/income.test.blind').readlines()]

In [91]:
# Predict on semi-blind test data.
blind_predictions(train_with_num, train_knn_targets, raw_blind,41)

In [93]:
with open("knn.income.test.predicted", "w") as output_file:
    for person in raw_blind:
        for features in person:
            output_file.write(str(features))
            if features != person[-1]:
                output_file.write(", ")
        output_file.write("\n")

### Averaged Perceptron Algorithm

In [116]:
# Training set, targets and mapping
train, targets, mapping = data('./data/income.train.txt.5k')

In [119]:
dev, dev_targets = data('./data/income.dev.txt')

In [186]:
blind, blind_targets = data('./data/income.test.blind')

In [131]:
def combo_features(train, mapping, col1, col2):
    new_train = train.copy()
    for f1 in mapping: 
        if f1[0] ==col1:
            for f2 in mapping: 
                if f2[0] == col2:
                    new_feats=((train[:,mapping[f1]] + train[:,mapping[f2]]) ==2).astype(np.int64)
                    new_train = np.concatenate((new_train, new_feats[:, np.newaxis]), axis=1)
    return new_train

In [132]:
# binarized train data with combination features of employment type status and COI
combo_train = combo_features(train, mapping, 4,8)

In [135]:
# binarized dev data with combination features of employment type status and COI
combo_dev = combo_features(dev,mapping,4,8)

In [189]:
combo_blind = combo_features(blind, mapping, 4,8)

In [160]:
# the evaluation on dev data.
def evaluation(dev_data, labels,weight):
    incorrect = 0
    positive = 0
    for feat, label in zip(dev_data, labels):
    # make the prediction
        pred = weight.dot(feat) 
        if pred > 0:
            positive += 1
        if pred * label <= 0:
            incorrect += 1
    err_rate = incorrect / len(labels) * 100
    pos_rate = positive / len(labels) * 100
    return err_rate, pos_rate

In [ ]:
# predict on dev data.
def test(devfile, model):
    tot, err = 0, 0
    predictions, pos, neg = [], {}, {}
    for i, (label, words) in enumerate(read_from(devfile), 1): # note 1...|D|
        err += label * (model.dot(make_vector(words))) <= 0
        pred = model.dot(make_vector(words))
        if label ==1:
            pos[i-1] = pred
        else:
            neg[i-1] = pred
        predictions.append(pred)
        
    return err/i ,predictions, pos, neg  # i is |D| now

In [179]:

def train_weights(train, targets, dev, dev_targets, n_epochs=5):
    t = time.time()
    weight = np.zeros(train.shape[-1])
    w_a = np.zeros(train.shape[-1])
    c = 1
    for it in range(n_epochs):
        updates = 0
        for i , (feat, label) in enumerate(zip(train, targets)):
            pred = weight.dot(feat)            
            if pred * label <= 0:
                # update the algorithm
                weight += label * feat
                w_a += label*feat * c
                updates += 1
            c +=1
        avg_perc = weight - w_a/c
        dev_err_rate, dev_pos_rate = evaluation(dev,dev_targets, avg_perc)
        print("epoch %d, update %.1f%%, dev %.1f%% " % (it, updates / i * 100, dev_err_rate))
    return avg_perc

In [181]:
best_model = train_weights(combo_train,targets, combo_dev, dev_targets, n_epochs=2)

epoch 0, update 25.4%, dev 14.8% 
epoch 1, update 24.2%, dev 14.0% 


In [207]:
# income.test.predicted with best perceptron algorithm(combo of occupation and COI)
def perc_predictions(binarized_data, rawfile, model):
    predictions = []
    for i, new_test in enumerate(binarized_data): 
        pred = model.dot(new_test) 
        if pred > 0:
            target = ">50K"
        else:
            target = "<=50K"
        rawfile[i].append(target)

In [208]:
# predict on semi-blind data
perc_predictions(combo_blind, raw_blind, best_model) #reinitialize raw blind file

In [209]:
# writing 
with open("perc.income.test.predicted", "w") as output_file:
    for person in raw_blind:
        for features in person:
            output_file.write(str(features))
            if features != person[-1]:
                output_file.write(", ")
        output_file.write("\n")

The k-nearest neighbors and averaged perceptron algorithm achieved 16.2% and 16.3% test error rate, respectively.